In [0]:
from pyspark.sql import functions as F

try:
    # Carregar tabelas silver
    df_pedidos = spark.table("medalhao.silver.ft_pedidos")
    df_consumidores = spark.table("medalhao.silver.ft_consumidores")
    df_itens = spark.table("medalhao.silver.ft_itens_pedidos")

    # Preparar fontes para o join
    
    # Localização do consumidor
    df_loc_consumidor = df_consumidores.select("id_consumidor", "cidade", "estado")
    
    # Dados de entrega do pedido
    df_dados_entrega = df_pedidos.select(
        "id_pedido",
        "id_consumidor",
        "entrega_no_prazo",
        "tempo_entrega_dias",
        "tempo_entrega_estimado_dias"
    )
    
    # Vendedor de cada pedido (pegando distintos para evitar duplicação se o mesmo vendedor tiver múltiplos itens no mesmo pedido)
    df_vendedores_pedido = df_itens.select("id_pedido", "id_vendedor").distinct()

    # Juntar as três fontes
    df_gold = df_dados_entrega.join(
        df_loc_consumidor,
        "id_consumidor",
        "left"
    ).join(
        df_vendedores_pedido,
        "id_pedido",
        "left"
    )
    
    # Selecionar colunas finais para a tabela fato 
    df_gold_final = df_gold.select(
        "id_pedido",
        "id_vendedor",
        "id_consumidor",
        "entrega_no_prazo",
        "tempo_entrega_dias",
        "tempo_entrega_estimado_dias",
        "cidade",
        "estado"
    )

    # Salvar na camada Gold
    df_gold_final.write.mode("overwrite").saveAsTable("medalhao.gold.ft_atrasos_pedidos_local_vendedor")
    
    print("tabela medalhao.gold.ft_atrasos_pedidos_local_vendedor criada.")

except Exception as e:
    print(f"erro em 2.1: {e}")

In [0]:
from pyspark.sql import functions as F

# Query SQL para criar a view 
query_view_2_2_1 = """
    CREATE OR REPLACE VIEW medalhao.gold.view_tempo_medio_entrega_localidade AS
    SELECT 
        cidade,
        estado,
        
        -- Média do tempo real
        AVG(tempo_entrega_dias) AS tempo_medio_entrega,
        
        -- Média do tempo estimado
        AVG(tempo_entrega_estimado_dias) AS tempo_medio_estimado,
        
        -- Indicador se a média real é maior que a estimada
        CASE
            WHEN AVG(tempo_entrega_dias) > AVG(tempo_entrega_estimado_dias) THEN 'SIM'
            ELSE 'NÃO'
        END AS entrega_maior_que_estimado
        
    FROM medalhao.gold.ft_atrasos_pedidos_local_vendedor
    WHERE tempo_entrega_dias IS NOT NULL -- Ignorar pedidos não entregues
    GROUP BY cidade, estado
"""

try:
    spark.sql(query_view_2_2_1)
    print("view medalhao.gold.view_tempo_medio_entrega_localidade criada.")
except Exception as e:
    print(f"erro em 2.2.1: {e}")

In [0]:
from pyspark.sql import functions as F

# Query SQL para criar a view
query_view_2_2_2 = """
    CREATE OR REPLACE VIEW medalhao.gold.view_vendedor_pontualidade AS
    SELECT 
        id_vendedor,
        
        -- Total de pedidos do vendedor
        COUNT(id_pedido) AS total_pedidos,
        
        -- Total de pedidos com atraso (onde entrega_no_prazo = 'Não')
        SUM(
            CASE 
                WHEN entrega_no_prazo = 'Não' THEN 1
                ELSE 0 
            END
        ) AS total_atrasados,
        
        -- Percentual de atraso (total_atrasados / total_pedidos)
        (
            SUM(
                CASE 
                    WHEN entrega_no_prazo = 'Não' THEN 1
                    ELSE 0 
                END
            ) / COUNT(id_pedido)
        ) * 100 AS percentual_atraso
        
    FROM medalhao.gold.ft_atrasos_pedidos_local_vendedor
    WHERE id_vendedor IS NOT NULL -- Ignorar pedidos sem vendedor
    GROUP BY id_vendedor
"""

try:
    spark.sql(query_view_2_2_2)
    print("view medalhao.gold.view_vendedor_pontualidade criada.")
except Exception as e:
    print(f"erro em 2.2.2: {e}")